In [2]:
import rasterio
import numpy as np

# Define the path to the input raster image
input_file = "lcz_predicted_mosaic.tif"

# Define the values from the left and right columns as lists
left_column = [2, 10, 5, 9, 7, 14, 13, 6, 11, 11, 1, 8, 12, 3, 0]
right_column = [14.53, 9.69, 7.11, 5.07, 4.34, 3.24, 2.10, 1.32, 0.91, 0.80, 0.63, 0.24, 0.00, 0.00, 0.00]

# Open the input raster image for reading
with rasterio.open(input_file) as src:
    # Read the first band from the input raster
    first_band = src.read(1)
    
    # Create a new array for the additional band
    new_band = np.zeros_like(first_band, dtype=np.float32)
    
    # Create a mapping dictionary from left_column values to right_column values
    mapping = dict(zip(left_column, right_column))
    
    # Iterate over the values in the first band
    for i, value in np.ndenumerate(first_band):
        # Find the corresponding value in the left column
        corresponding_value = mapping.get(value)
        
        # Assign the corresponding value from the right column to the corresponding pixel in the new band
        if corresponding_value is not None:
            new_band[i] = corresponding_value
    
    # Update the metadata to indicate the addition of the new band
    metadata = src.meta
    metadata['count'] = 2  # Update the band count
    
    # Create a new file with the updated band
    output_file = "lcz_feature_importance.tif"
    with rasterio.open(output_file, 'w', **metadata) as dst:
        # Write the original first band
        dst.write_band(1, first_band)
        
        # Write the new band as the second band
        dst.write_band(2, new_band)
